# Setup

In [56]:
from parameters.parameters import prune_quantiles, prune_measure

debug = True # If True, will produce plots and/or verbose output to double-check

import sys
sys.path.append("..")

# Path
print("Current working directory: ", sys.path[0])
%run -i "../parameters/parameters.py"
%run -i "../src/path.py"
%run -i "../src/setup.py"
%run -i "../src/functions.py"
%run -i "greedy_triangulation.py"
%run -i "GraphSimilarity.py"

%load_ext watermark
%watermark -n -v -m -g -iv

Current working directory:  /Users/jaborup/Projects/continuous-bike-growth
Loaded parameters.

Loaded PATH.



=== Cities ===
{   'montreal': {   'countryid': 'can',
                    'name': 'Montreal',
                    'nominatimstring': 'montreal (06)'}}


Setup finished.

Loaded functions.

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
osgeo.gdal 3.8.5
csv        1.0
numpy      1.26.4
geojson    2.5.0
matplotlib 3.8.4
shapely    2.0.4
pyproj     3.6.1
fiona      1.9.5
networkx   3.3
igraph     0.11.6
osmnx      2.0.0b3
pandas     2.2.3
watermark  2.0.2
geopandas  1.0.1
Thu Nov 14 2024 

CPython 3.12.7
IPython 8.6.0

compiler   : Clang 17.0.6 
system     : Darwin
release    : 23.6.0
machine    : arm64
processor  : arm
CPU cores  : 8
interpreter: 64bit
Git hash   : fb1f59bcbafce3c90f905497b55f1c69c21488c3


# Run the program

In [57]:
for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    print(placeid + ": Generating networks")

    # Load networks
    G_carall = csv_to_ig(PATH["data"] + placeid + "/", placeid, 'carall')

    # Load POIs
    with open(PATH["data"] + placeid + "/" + placeid + '_poi_' + poi_source + '_nnidscarall.csv') as f:
        nnids = [int(line.rstrip()) for line in f]

    prune_quantiles = [0.25, 0.5, 0.75, 1]

    # Generation
    gts, abstract_gts = greedy_triangulation_in_steps(
        graph=G_carall,
        pois=nnids,
        subgraph_percentage=0.6,
        prune_quantiles=prune_quantiles,
        prune_measure="betweenness",
    )
    #
    # filename = placeid + '_poi_' + poi_source + "_" + prune_measure
    # result_file = open(PATH["results"] + placeid + "/" + filename + ".pickle", 'rb')
    # res = pickle.load(result_file)
    # szell_gts = res["GTs"]

    # results = {
    #     "placeid": placeid,
    #     "prune_measure": prune_measure,
    #     "poi_source": poi_source,
    #     "prune_quantiles": prune_quantiles,
    #     "GTs": gts,
    #     "GT_abstracts": abstract_gts,
    #     # "MST": MST,
    #     # "MST_abstract": MST_abstract
    # }
    # write_result(results, "pickle", f'{placeid}_research', poi_source, prune_measure, ".pickle")

    (GTs, GT_abstracts) = _greedy_triangulation_routing(G_carall, nnids, prune_quantiles, "betweenness")

    # results = {
    #     "placeid": placeid,
    #     "prune_measure": prune_measure,
    #     "poi_source": poi_source,
    #     "prune_quantiles": prune_quantiles,
    #     "GTs": GTs,
    #     "GT_abstracts": GT_abstracts,
    #     # "MST": MST,
    #     # "MST_abstract": MST_abstract
    # }
    # write_result(results, "pickle", f'{placeid}_szell', poi_source, prune_measure, ".pickle")

    for prune_quantile, gt, szell_gt in zip(prune_quantiles, gts, GTs):
        print(f'Number of vertices in research project: {gt.vcount()}')
        print(f"Number of vertices in Szell's project: {szell_gt.vcount()}")

        print(f"Prune quantile: {prune_quantile}")
        error, subgraph_disconnected_points, graph_disconnected_points = sum_of_errors_pois(gt, szell_gt, nnids)
        print("Error: ", error)
        print("Disconnected points in subgraph: ", subgraph_disconnected_points)

Cities:   0%|          | 0/1 [00:00<?, ?it/s]

montreal: Generating networks


/Users/jaborup/Projects/continuous-bike-growth/src/functions.py:345: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  n = pd.read_csv(p + prefix + '_nodes.csv')
/Users/jaborup/Projects/continuous-bike-growth/src/functions.py:346: DtypeWarning: Columns (5,9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  e = pd.read_csv(p + prefix + '_edges.csv')


Greedy triangulation on 60.0% subgraph:   0%|          | 0/4 [00:00<?, ?it/s]

Vertex in GT_abstract: True
Vertex in GT_abstract after GT: True
All POI IDs: [31706396, 32124695, 32124745, 32663000, 116346943, 209365112, 209451593, 209493424, 213462304, 213784072, 213839168, 213962625, 215207996, 215665097, 215694709, 218225504, 218315575, 218344316, 220784679, 220785326, 220840262, 224688993, 224826175, 224843617, 225015278, 226048799, 241888446, 243075442, 245474360, 246093663, 246280405, 246282168, 246283371, 246285415, 246453410, 246456569, 246464012, 246653218, 246653275, 246659329, 246665007, 246665166, 246713243, 246713656, 246714196, 248512113, 248512751, 274284366, 276648428, 282710465, 294245158, 295078610, 295801587, 300060744, 315995923, 316169589, 323035549, 344876938, 412410174, 437715910, 437862509, 459390315, 472502195, 534470749, 537195425, 537213947, 581098365, 581118908, 595748514, 595748527, 595753065, 595756445, 596515729, 596568927, 596570617, 596571405, 596643209, 596643906, 597038571, 598409708, 604059672, 604059741, 604060255, 604060791, 6

Vertex in pruned_graph after GT: True
Pruning betweenness...
Finished pruning betweenness
Vertex in pruned_graph after pruning: True
Edge: 209493424 -> 215665097
Edge: 5412368812 -> 9761716172
Edge: 534470749 -> 537195425
Edge: 215665097 -> 215694709
Edge: 604060791 -> 604130580
Edge: 596643209 -> 598409708
Edge: 246653218 -> 11160898053
Edge: 218225504 -> 11160898053
Edge: 31706396 -> 32124695
Edge: 248512113 -> 248512751
Vertex in GT_indices: False
Vertex in GT: False


Greedy triangulation:   0%|          | 0/4 [00:00<?, ?it/s]

Pruning betweenness...
Finished pruning betweenness
Pruning betweenness...
Finished pruning betweenness
Pruning betweenness...
Finished pruning betweenness
Pruning betweenness...
Finished pruning betweenness
Number of vertices in research project: 9645
Number of vertices in Szell's project: 12687
Prune quantile: 0.25
Graph 1 contains 4483790348: False
Graph 2 contains 4483790348: False
Expecting 10153.0 iterations
Iteration 0
v: 653993473, w: 11160898053
v: 653993473, w: 4483790348


ValueError: no such vertex

In [ ]:
for placeid, placeinfo in tqdm(cities.items(), desc="Cities"):
    result_file = open(PATH["results"] + placeid + "/" + filename + ".pickle", 'rb')
    res = pickle.load(result_file)
    szell_gts = res["GTs"]